In [3]:
import pandas as pd
import numpy as np

import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import multiprocessing.popen_spawn_win32

#create DASK local cluster and client
cluster = LocalCluster(n_workers=4)
client = Client(cluster)

#read csv file
bat_info = dd.read_csv('../data/batt_info.csv000',
                       delimiter ="\1",
                       assume_missing=True)

#drop missing value
bat_info_clean = bat_info[bat_info.interval_start_utc <'2022']
bat_info_clean = bat_info_clean[(bat_info.interval_end_utc <'2022')& (bat_info.interval_end_utc >'2019')]
bat_info_clean = bat_info_clean[(bat_info_clean.dt <'2022')& (bat_info_clean.dt >'2019')]
bat_info_clean = bat_info_clean[bat_info_clean.battery_enum <10]

#Keep only one record for one guid
bat_info_clean = bat_info_clean.drop_duplicates(subset=['guid'])

#Normalize the chemistry
bat_info_clean['chemistry'] = bat_info_clean['chemistry'].map_partitions(lambda x: x.astype(str).str.lower())

def norm_chem(i):
    #find the value for lion
    
    s = i.chemistry
    if(s == 'lion'):
        return 'lion'
    elif(s == 'liio'):
        return 'lion'
    elif(s == 'li-i'):
        return 'lion'
    elif(s == 'li'):
        return 'lion'
    elif(s == 'lio'):
        return 'lion'
    
    #find the value for lip
    elif(s == 'lip'):
        return 'lip'
    elif(s == 'li p'):
        return 'lip'
    elif(s == 'lipo'):
        return 'lip'
    
    #find the value of pbac
    elif(s == 'pbac'):
        return 'pbac'
    
    #find the missing values
    elif(s == 'nan'):
        return np.nan
    elif(s == 'miss'):
        return np.nan
    else:
        return np.nan
    
bat_info_clean['chemistry'] = bat_info_clean.apply(norm_chem, axis=1, meta=(None, 'str'))

#drop missing value
bat_info_clean = bat_info_clean.dropna(subset=['chemistry','battery_count','designed_capacity'])
cond1 = bat_info_clean.designed_capacity != 0
cond2 = bat_info_clean.battery_count != 0

bat_info_clean = bat_info_clean[cond1& cond2]

#only keep useful features
bat_info_clean =bat_info_clean[['chemistry','battery_count','designed_capacity']]
bat_info_clean = bat_info_clean.compute()

#save the output as csv file
bat_info_clean.to_csv('../data/batt_info_clean.csv',index = False)

d:\python\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 6281 instead
  warnings.warn(


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/batt_info_clean.csv')
len(df)

32631